# 02 - Self-Consistency

This notebook demonstrates self-consistency prompting: running the same LLM call multiple times and aggregating results to get more reliable sentiment analysis.

**Prerequisites:** Set `INSERT_API_KEY_HERE` before running.

In [ ]:
from openai import AzureOpenAI, OpenAI
import os
from pydantic import BaseModel, Field
from enum import Enum
from typing import List
from collections import defaultdict, Counter

client = OpenAI(api_key="INSERT_API_KEY_HERE")

## Define Structured Output Models

Pydantic models for bank sentiment analysis with constrained sentiment values.

In [ ]:
class Sentiment(str, Enum):
    positive = "positive"
    neutral = "neutral"
    negative = "negative"


class BankSentiment(BaseModel):
    bank_name: str = Field(description="Name of the bank.")
    sentiment: Sentiment = Field(description="Sentiment towards the bank.")
    sentiment_explanation: str = Field(description="Justification of the chosen sentiment.")


class NewsAnalysis(BaseModel):
    bank_sentiments: List[BankSentiment] = Field(description="Sentiment analysis for each mentioned bank.")

## Article & Prompts

A synthetic news article with deliberately ambiguous sentiment signals for three fictional banks.

In [ ]:
doc_article = """
In a week of turbulent trading, three mid-sized lenders \u2014 Aurora Capital Bank, GreenStone Community Bank, and Horizon Meridian Bank \u2014 found themselves at the center of conflicting analyst commentary.

Aurora Capital Bank surprised markets with stronger-than-expected quarterly earnings, helped by robust fee income and tight cost control. Several analysts praised its \u201cdisciplined risk management\u201d and \u201cimpressive digital onboarding experience,\u201d noting that customer satisfaction scores hit record highs. However, regulators have quietly raised concerns about Aurora\u2019s growing exposure to speculative commercial real estate, warning that \u201ca downturn could quickly erode its currently healthy capital buffers.\u201d Some investors remain wary, describing the bank\u2019s growth strategy as \u201caggressive and potentially reckless\u201d despite the upbeat headline numbers.

GreenStone Community Bank, which focuses on sustainable lending and local businesses, announced a modest profit decline. Management framed the results as a \u201cnecessary short-term sacrifice\u201d to expand green lending programs and offer temporary fee waivers to struggling small firms. Environmental groups applauded GreenStone\u2019s \u201cgenuine commitment to social impact,\u201d and local entrepreneurs credit the bank with \u201ckeeping their doors open.\u201d Yet the stock fell after an influential brokerage downgraded GreenStone to \u201cunderperform,\u201d citing \u201cweak profitability, slow digital transformation, and a lack of operational discipline.\u201d While long-term customers remain loyal, some shareholders question whether the bank is \u201ctoo idealistic to compete effectively.\u201d

Horizon Meridian Bank delivered mixed results, beating expectations on revenue but missing consensus on net income due to one-off restructuring charges. Executives called the quarter a \u201cturning point,\u201d emphasizing early success in reducing non-performing loans and modernizing back-office systems. A major rating agency reaffirmed Horizon\u2019s investment-grade status and highlighted its \u201cimproving risk profile.\u201d At the same time, employee unions criticized recent branch closures as \u201cshort-sighted cost cutting,\u201d and social media sentiment turned sharply negative after reports of extended call-center wait times. While a few market commentators described Horizon as \u201cquietly fixing its problems,\u201d others warned that \u201cmanagement credibility is still fragile and customer trust far from restored.\u201d

Overall, investors and observers are sharply divided on all three institutions: each bank can point to clear signs of progress and stability, yet each is also dogged by lingering doubts, conflicting signals, and narratives that can be read as either cautiously optimistic or quietly alarming.
"""

system_prompt = """
You are a senior financial text analysis engine.

Constraints:
- Focus on banks only, ignore other companies.
- Be conservative with positive sentiment; require clearly favorable language.
"""

user_prompt = f"""
Objectives:
- Extract all banks mentioned in a news article.
- Determine sentiment (positive / neutral / negative) towards each bank.

Article:
{doc_article}
"""

## Single Run

A single LLM call to see the baseline result.

In [ ]:
completion = client.beta.chat.completions.parse(
    model="gpt-4.1",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    response_format=NewsAnalysis,
)
response = completion.choices[0].message.parsed

for bs in response.bank_sentiments:
        print(f"{bs.bank_name}: {str(bs.sentiment)}")

## Self-Consistency: Multiple Runs

Run the same prompt N times and aggregate the sentiment counts per bank to see how consistent the model is.

In [ ]:
n_runs = 10
sentiment_counts = defaultdict(Counter)

for _ in range(n_runs):
    completion = client.beta.chat.completions.parse(
        model="gpt-4.1",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        response_format=NewsAnalysis,
    )

    analysis: NewsAnalysis = completion.choices[0].message.parsed

    for bs in analysis.bank_sentiments:
        bank = bs.bank_name.strip()
        sentiment_counts[bank][bs.sentiment.value] += 1

for bank, counts in sentiment_counts.items():
    print(f"Bank: {bank}")
    for sentiment, count in counts.items():
        print(f"  {sentiment}: {count}")